In [2]:
# Based on spotify hits list we would like to check if there is a way to predict music hit based on provided features.

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error


#it's good idea to set bigger plot size at a beging
sns.set(rc = {'figure.figsize':(23,8)})

In [3]:
df = pd.read_csv('../input/top-hits-spotify-from-20002019/songs_normalize.csv')
df.head()

In [4]:
# first We need to check if there is any missing values
df.isna().sum()

In [5]:
# Have a broader look on data 

df.describe()

In [6]:
# Let's have a look on the most popular artist and music genre
df['artist'].value_counts().head(20)

In [7]:
df['genre'].value_counts().head(20)

In [8]:
# As We can see there is a lot of songs with more than one genre. It would be good idea to split this
# First We need to find out what is a maximum number of genres per song. 

genres = df['genre']
len(genres.str.findall(',').max())

In [9]:
# So if there is maximum 3 comas than there is maximum 4 genres per song.

df[['genre_1','genre_2','genre_3','genre_4']] = df['genre'].str.split(',',expand=True)

In [10]:
# We can also add number of spicified genres as another feature

df['number_of_genre'] = df['genre'].apply(lambda x: x.count(',')+1)

In [12]:
genres_unique =[]
genre_list = ['genre_1','genre_2','genre_3','genre_4']
for x in genre_list:
    genres_unique.append(df['genre_1'].unique())
    

In [13]:
genres_unique



In [13]:
df_genre.explode('genre')

In [30]:
#Let's have a look on distrubution of tracks across 2000-2019
df_years = df['year'].value_counts()
sns.barplot(df_years.index,df_years.values)
plt.show()

In [10]:

#Let's have a look on distrubution of popularity
df_popularity = df['popularity'].value_counts()
sns.barplot(df_popularity.index,df_popularity.values)
plt.show()

In [11]:
df['popularity'].describe()


In [43]:
# We would like to predict the popularity of track. It is a classification problem - but for this data it would be good idea to group this target class to four class based on popularity values.
# Let's set Platinium(top 25%),Gold(25%-50%), Silver(50%-75%),Bronze(Button 25%) label on this dataset

custom_label= ['4','3','2','1']

df['popularity_label'] = pd.qcut(df['popularity'],q=4,labels=custom_label)
df.head()

In [54]:
df = df.astype({"popularity_label": int})
df['genre_1'].get_dummies()

In [53]:
sns.heatmap(df.corr(),annot=True);

In [25]:
X =df.select_dtypes(include=np.number)
y =df['popularity_label']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state=42)
lr_model = LogisticRegression()
lr_model.fit(X_train,y_train)
y_pred = lr_model.predict(X_test)

score = lr_model.score(X_test,y_test)
score

In [22]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Train the model using the training sets

model.fit(X_train,y_train)


#Predict Output

predicted= model.predict(X_test) # 0:Overcast, 2:Mild

from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, predicted))

In [23]:
from sklearn import svm


model = svm.SVC(kernel='linear') 


label = df['popularity_label']
X =df[['loudness','speechiness']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Train the model using the training sets

model.fit(X_train,y_train)


#Predict Output

predicted= model.predict(X_test) # 0:Overcast, 2:Mild

from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, predicted))

In [31]:
df.head()

In [42]:
ax = sns.swarmplot(data=df, x="popularity_label", y="speechiness", hue="popularity_label")
ax.set(ylabel="")